In [1]:
# Use the Elhub API to retrieve hourly production data for all price areas using PRODUCTION_PER_GROUP_MBA_HOUR for all days and hours of the year 2021.
# The API allows retrieving different datasets for each entity using the following format: https://api.elhub.no/energy-data/v0/{entity}?dataset={dataset}
# The API returns data in JSON format. You can use the requests library to make the API calls and the pandas library to process the data.
# You can use the following code to get started:
import requests
import pandas as pd
import json
import datetime as dt
import matplotlib.pyplot as plt
# Define the API endpoint and parameters
endpoint = "https://api.elhub.no/energy-data/v0/price-areas"
dataset = "PRODUCTION_PER_GROUP_MBA_HOUR"
start_date = "2021-01-01T00:00:00Z"
end_date = "2021-12-31T23:00:00Z"
# Make the API call
url = f"{endpoint}?dataset={dataset}&start={start_date}&end={end_date}"
response = requests.get(url)
print(f"Status code: {response.status_code}")
data = response.json()



Status code: 200


In [2]:
data = response.json()

# Print top-level keys
print("Top-level keys:", list(data.keys()))

# Print first element of 'data' if it exists
if "data" in data:
    print("First entry keys:", data["data"][0].keys())
    print("Attributes keys:", data["data"][0]["attributes"].keys())
else:
    print("No 'data' key in response")


Top-level keys: ['data', 'links', 'meta']
First entry keys: dict_keys(['attributes', 'id', 'type'])
Attributes keys: dict_keys(['country', 'eic', 'name', 'productionPerGroupMbaHour'])


In [3]:
all_records = []

for entry in data.get("data", []):
    attrs = entry.get("attributes", {})
    recs = attrs.get("productionPerGroupMbaHour", [])
    # Filter out placeholder "*"
    recs = [r for r in recs if r.get("productionGroup") != "*"]
    all_records.extend(recs)

print(f"Total records collected: {len(all_records)}")


Total records collected: 17150


In [4]:
df = pd.DataFrame(all_records)

# Convert timestamps and numeric columns
df['startTime'] = pd.to_datetime(df['startTime'])
df['quantityKwh'] = pd.to_numeric(df['quantityKwh'])

# Optional: set startTime as index
df.set_index('startTime', inplace=True)

df.head()


,endTime,lastUpdatedTime,priceArea,productionGroup,quantityKwh
startTime,,,,,
2025-09-17 10:00:00+02:00,2025-09-17T11:00:00+02:00,2025-10-01T18:00:10+02:00,NO1,hydro,2580360.8
2025-09-17 11:00:00+02:00,2025-09-17T12:00:00+02:00,2025-10-01T18:00:10+02:00,NO1,hydro,2619874.8
2025-09-17 12:00:00+02:00,2025-09-17T13:00:00+02:00,2025-10-01T18:00:10+02:00,NO1,hydro,2442582.0
2025-09-17 13:00:00+02:00,2025-09-17T14:00:00+02:00,2025-10-01T18:00:10+02:00,NO1,hydro,2451032.2
2025-09-17 14:00:00+02:00,2025-09-17T15:00:00+02:00,2025-10-01T18:00:10+02:00,NO1,hydro,2446066.0


In [5]:
print(df['priceArea'].unique())
print(df['productionGroup'].unique())
print(df.index.min(), df.index.max())


['NO1' 'NO2' 'NO3' 'NO4' 'NO5']
['hydro' 'other' 'solar' 'thermal' 'wind']
2025-09-17 10:00:00+02:00 2025-10-15 23:00:00+02:00


In [ ]:
def page_extra() -> None:
    st.title("Production Data by Price Area")

    col1, col2 = st.columns(2)

    # Left column: select price area
    with col1:
        price_areas = collection.distinct("pricearea")  # get all unique price areas
        selected_area = st.radio("Select Price Area", price_areas)

    # Right column: show pie chart for selected price area
    with col2:
        # Fetch data from MongoDB
        data = list(collection.find({"pricearea": selected_area}))
        if data:
            df_area = pd.DataFrame(data)
            df_grouped = df_area.groupby("productiongroup")["quantitykwh"].sum().reset_index()

            import plotly.express as px
            fig = px.pie(
                df_grouped, 
                values="quantitykwh", 
                names="productiongroup",
                title=f"Total Production for {selected_area}"
            )
            st.plotly_chart(fig)
        else:
            st.write("No data found for this price area.")
